In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Input, Activation, add, Add, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.models import Sequential, Model
from keras import optimizers

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

In [ ]:
IMG_WIDTH = 224
IMG_HEIGHT = 224
CHANNELS = 3
CLASSES = 18
batch_size = 64
train_dir = './TrainCropped/'
test_dir = './TestCropped/'

In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=90,
    fill_mode='constant',
    cval=0xCC,
    horizontal_flip=True,
    vertical_flip=True,
    channel_shift_range=100,
    validation_split=0.2
    )

test_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size=(IMG_WIDTH,IMG_HEIGHT),
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=True,
            subset='training'
        )

In [ ]:
validation_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size=(IMG_WIDTH,IMG_HEIGHT),
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=False,
            subset='validation'
        )

In [ ]:
test_generator = test_datagen.flow_from_directory(
            test_dir,
            target_size=(IMG_WIDTH,IMG_HEIGHT),
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=False
)

In [ ]:
model_ENB0 = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(IMG_WIDTH,IMG_HEIGHT,CHANNELS))
model_ENB0.trainable = False
model_ENB0.summary()

In [ ]:
model = Sequential()
model.add(model_ENB0)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.2))
model.add(Dense(CLASSES,activation="softmax"))
model.summary()

In [ ]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = optimizers.Adam(learning_rate=1e-2),
              metrics=['accuracy'])
 
STEP_SIZE_TRAIN=train_generator.n // train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n  // validation_generator.batch_size

epochs = 10

history = model.fit_generator(train_generator,
                             steps_per_epoch=STEP_SIZE_TRAIN,
                             epochs=epochs,
                             
                             validation_data=validation_generator,
                             validation_steps=STEP_SIZE_VALID)

In [ ]:
scores = model.evaluate(test_generator, verbose=1)

In [ ]:
model_ENB0.trainable = True
set_trainable = False

for layer in model_ENB0.layers:
    if layer.name == 'block6d_se_excite':
        set_trainable = True
    if set_trainable:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = True
        else:
            layer.trainable = False
    else:
        layer.trainable = False
        
model.summary()

In [ ]:
# Fine Tuning
model.compile(loss = 'categorical_crossentropy',
              optimizer = optimizers.Adam(learning_rate=1e-4),
              metrics=['accuracy'])
 
STEP_SIZE_TRAIN=train_generator.n // train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n  // validation_generator.batch_size

epochs = 5

history = model.fit_generator(train_generator,
                             steps_per_epoch=STEP_SIZE_TRAIN,
                             epochs=epochs,
                             validation_data=validation_generator,
                             validation_steps=STEP_SIZE_VALID)

In [ ]:
scores = model.evaluate(test_generator, verbose=1)

In [ ]:
model.save('cats_18_EfficientNetB0.h5')

In [ ]:
CAT_BREEDS = [
               'Бенгальская', 'Бомбейская', 'Британская короткошерстная',
               'Бурмилла', 'Девон-рекс', 'Европейская короткошерстная',
               'Экзотическая короткошерстная', 'Мейн-кун', 'Нибелунг',
               'Персидская', 'Рэгдолл', 'Русская голубая',
               'Шотландская вислоухая', 'Сибирская', 'Сингапурская',
               'Сомалийская', 'Сфинкс', 'Черепаховая'
            ]

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

In [ ]:
image = load_img('cat.png', target_size=(224, 224))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
pred = model.predict(image)
pred = np.argmax(pred)
print(CAT_BREEDS[pred])